In [1]:
import pandas as pd

In [2]:
articles = pd.read_json("articles_from_all_used_journals_ss.pd")

In [3]:
len(articles)

1401513

In [4]:
def extract_authors(authors):
    return [d["name"] for d in authors]

articles.loc[0,:]

title            Cupular sense organs in Ciona (Tunicata: Ascid...
paperAbstract    The cupular organs of the atrial (exhalent) si...
authors          [{'name': 'Q.  Bone', 'ids': ['7010725']}, {'n...
doi                             10.1111/J.1469-7998.1978.TB03931.X
year                                                        2009.0
journalVolume                                                  186
journalPages                                               417-429
fieldsOfStudy                                            [Biology]
journal_name                                    Journal of Zoology
Name: 0, dtype: object

In [21]:
import json
with open("journal_name_to_standard.json") as f:
    st = json.load(f)

In [18]:
st

{'the journal of applied ecology': 'journal of applied ecology',
 'agriculture ecosystems and environment': 'agriculture, ecosystems & environment',
 'agriculture, ecosystems and environment': 'agriculture, ecosystems & environment',
 'restoration ecology': 'restoration ecology',
 'the journal of wildlife management': 'journal of wildlife management',
 'waterbirds: the international journal of waterbird biology': 'waterbirds',
 'animal conservation': 'animal conservation',
 'vegetatio': 'vegetatio',
 'ices journal of marine science: journal du conseil': 'ices journal of marine science',
 'the journal of animal ecology': 'journal of animal ecology',
 'journal of insect conservation': 'journal of insect conservation',
 'acta oecologica': 'acta oecologica',
 'plos one': 'plos one',
 'the journal of ecology': 'journal of ecology',
 'mammalian biology - zeitschrift für säugetierkunde': 'mammalian biology',
 'animal welfare': 'animal welfare',
 'mammalia': 'mammalia',
 'knowledge and managem

In [22]:
from sources.databases.db_definitions import DBArticleMetadata
from sources.databases.internal_databases import SQLiteDB
from sources.databases.article_data_db import ArticleRepositoryAPI
from sources.databases.journal_name_issn_database import JournalNameIssnDatabase
from datetime import date

def map_row_to_db_entry(row, issn_db: JournalNameIssnDatabase):
     jn = row.journal_name
    
    if jn.lower() in st:
        jn = st[jn.lower()]
    issn = issn_db.get_issn_from_name(jn)
    

    
    return DBArticleMetadata(title=row.title,
                             authors=extract_authors(row.authors),
                             doi=row.doi,
                             publication_date=row.year,
                             abstract=row.paperAbstract,
                             journal_name=jn,
                             journal_volume=row.journalVolume,
                             repo_identifier="sem_scholar",
                             issn=issn,
                             url=f"www.doi.org/{row.doi}",
                             sync_date=date.today().isoformat(),
                             )

In [15]:
with JournalNameIssnDatabase() as db:
    res = (map_row_to_db_entry(articles.loc[0,:], db))
    print(res)


 
 Title: Cupular sense organs in Ciona (Tunicata: Ascidiacea) 
            
 Authors: Q.  Bone, K. P. Ryan 
            
 Published: 2009.0 
 DOI: 10.1111/J.1469-7998.1978.TB03931.X
 Abstract: 
 The cupular organs of the atrial (exhalent) siphon of Ciona have been studied
with scanning and transmission microscopy and are shown to resemble those of the
vertebrate acoustico-lateralis system in several respects. The sensory cells are
ciliated, and their cilia are apparently non-motile, having a modified inner
tubular array. These cells lie amongst supporting cells that probably secrete
the cupula, which is composed of polysaccharide and proteins as is the test.
Ciona is sensitive to near-field vibrations, even after the brain has been
removed; the significance of this observation and of the arrangement of the
cupular organs is discussed. It is concluded that the tunicates show a suitable
morphological starting point for the vertebrate acoustico-lateralis system. 




In [23]:
with ArticleRepositoryAPI(SQLiteDB()) as article_db:
    with JournalNameIssnDatabase() as issn_db:
        for ind, row in articles.iterrows():
             res = (map_row_to_db_entry(row, issn_db))
             article_db.store_article(res)


In [24]:
print("Hello")

Hello
